# Proportional Election Simulations

**Prepare Workspace**

In [2]:
# Import libraries and set appearance parameters
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import datasets
df = pd.read_csv('Election Results.csv', thousands=',')
df_sum = pd.read_csv('Summary.csv', thousands=',')

# Declare global variables
pt = list(df['Province/Territory'].unique())

# Hide extra warnings
import warnings
warnings.filterwarnings(action='ignore')

**1. Dual Member Proportional**

In [53]:
# Build new dataframe
cols = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other', 'Turnout', 'Area']
df_dmp = pd.DataFrame(columns=cols)

# Build new ridings
for i in pt:
    prov = df[df['Province/Territory'] == i][cols]
    dual = df[df['Province/Territory'] == i][df['Area'] <= 1500][cols]
    
    if len(dual) % 2 == 1:
        dual = dual.sort_values('Area').drop(dual.sort_values('Area').tail(1).index)
    
    single = prov[~prov.isin(dual)].dropna(how='all')
    combined = dual.groupby(np.arange(len(dual))//2).sum()
    combined['System'], single['System'] = 'Combined', 'Single'
    combined['PT'], single['PT'] = i, i
    
    df_dmp = df_dmp.append([combined, single], sort=False).fillna(0)

# Calculate percentage of votes
parties = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other']
for i in parties:
    df_dmp['%' + i] = round(df_dmp[i] / df_dmp['Turnout'] * 100, 1)

# Calculate first seat winners
df_dmp['Highest'] = df_dmp[parties].max(axis=1)
df_dmp['Highest Party'], df_dmp['First Seat'] = np.NaN, np.NaN

for i in parties:
    df_dmp['Highest Party'] = np.where(df_dmp['Highest'] == df_dmp[i], i, df_dmp['Highest Party'])
df_dmp['First Seat'] = df_dmp['Highest Party']

# Calculate second seat winners
for i in parties:
    df_dmp[i + ' Round 2'] = np.where(df_dmp['First Seat'] == i, df_dmp[i]/2, df_dmp[i])

round2 = ['LPC Round 2', 'CPC Round 2', 'BQ Round 2', 'NDP Round 2', 'GP Round 2', 'PP Round 2', 'Other Round 2']
df_dmp['Highest 2'] = df_dmp[round2].max(axis=1)
df_dmp['Highest Party 2'], df_dmp['Second Seat'] = np.NaN, np.NaN
for i in round2:
    df_dmp['Highest Party 2'] = np.where(df_dmp['Highest 2'] == df_dmp[i], i, df_dmp['Highest Party 2'])
    
df_dmp['Second Seat'] = np.where(df_dmp['System'] == 'Combined', df_dmp['Highest Party 2'], df_dmp['Second Seat'])
df_dmp['Second Seat'] = df_dmp['Second Seat'].str[:-8]

# Count total seats
seat1 = pd.DataFrame(df_dmp['First Seat'].value_counts())
seat2 = pd.DataFrame(df_dmp['Second Seat'].value_counts())
total = seat1.join(seat2).fillna(0)
total['Total'] = total['First Seat'] + total['Second Seat']

**2. MMP**

In [134]:
# Build new dataframe
cols = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other', 'Turnout', 'Area']
df_dmp = pd.DataFrame(columns=cols)
df_mmp = pd.DataFrame(columns=cols)

# Build new ridings
for i in pt:
    prov = df[df['Province/Territory'] == i][cols]
    dual = df[df['Province/Territory'] == i][df['Area'] <= 1500][cols]
    
    if len(dual) % 2 == 1:
        dual = dual.sort_values('Area').drop(dual.sort_values('Area').tail(1).index)
    
    single = prov[~prov.isin(dual)].dropna(how='all')
    combined = dual.groupby(np.arange(len(dual))//2).sum()
    combined['System'], single['System'] = 'Combined', 'Single'
    combined['PT'], single['PT'] = i, i
    
    df_dmp = df_dmp.append([combined, single], sort=False).fillna(0)

df_dmp = df_dmp.reset_index(drop=True)
cols = list(df_dmp.columns)
for i in pt:
    prov = df_dmp[df_dmp['PT'] == i][cols]
    quad = df_dmp[df_dmp['PT'] == i][df_dmp['Area'] <= 1500][df_dmp['System'] == 'Combined'][cols]
    
    if len(quad) % 2 == 1:
        quad = quad.sort_values('Area').drop(quad.sort_values('Area').tail(1).index)
    
    nonquad = prov[~prov.isin(quad)].dropna(how='all')
    double_combined = quad.groupby(np.arange(len(quad))//2).sum()
    double_combined['System'], double_combined['PT'] = 'Double Combined', i
    
    df_mmp = df_mmp.append([double_combined, nonquad], sort=False).fillna(0)

# Calculate percentage of votes
parties = ['LPC', 'CPC', 'BQ', 'NDP', 'GP', 'PP', 'Other']
for i in parties:
    df_mmp['%' + i] = round(df_mmp[i] / df_mmp['Turnout'] * 100, 1)
    


In [131]:
#(len(df_mmp[df_mmp['System'] == 'Double Combined']) * 4) + (len(df_mmp[df_mmp['System'] == 'Combined']) * 2) + len(df_mmp[df_mmp['System'] == 'Single'])

338

**3. URP**

In [3]:
df.head()

,Province/Territory,Riding,LPC,CPC,BQ,NDP,GP,PP,Other,Turnout,Successful Voters,% Successful,Unsuccessful Voters,% Unsuccessful,Elected,% LPC,Population,Electors,Area
0,AB,Banff-Airdrie,8216,54580,NaN,7960,3230,2609.0,NaN,76595,54580,71.30%,22015,28.70%,CPC,10.70%,105442,87666,12282
1,AB,Battle River-Crowfoot,2485,52497,NaN,3089,1750,1583.0,NaN,61404,52497,85.50%,8907,14.50%,CPC,4.00%,107140,79873,53072
2,AB,Bow River,3104,45691,NaN,3010,805,1302.0,445.0,54357,45691,84.10%,8666,15.90%,CPC,5.70%,103871,73598,24427
3,AB,Calgary Centre,17199,36360,NaN,6290,2723,867.0,495.0,63934,36360,56.90%,27574,43.10%,CPC,26.90%,108931,83497,49
4,AB,Calgary Confederation,14462,35648,NaN,7113,5488,1117.0,517.0,64345,35648,55.40%,28697,44.60%,CPC,22.50%,111785,85997,54


In [19]:
df_sum

,Party,Party Votes,Total Votes,Percent of Total Votes,Elected,Seats if Proportional
0,LPC,5911617.0,17873038.0,33.1%,157.0,112.0
1,CPC,6153341.0,17873038.0,34.4%,121.0,116.0
2,BQ,1377205.0,17873038.0,7.7%,32.0,26.0
3,NDP,2843661.0,17873038.0,15.9%,24.0,54.0
4,GP,1157758.0,17873038.0,6.5%,3.0,22.0
5,PP,292802.0,17873038.0,1.6%,0.0,6.0
6,Other,136654.0,17873038.0,0.8%,1.0,3.0
7,NaN,NaN,NaN,NaN,NaN,NaN
